In [ ]:
"""
torch.Generator() 本身只是一个随机数生成器对象，不会产生随机数。
要想用它产生稳定的随机序列，需要配合 .manual_seed(seed) 设置种子。
它的好处是你可以为不同任务准备不同的随机流，互不干扰，比 torch.manual_seed()（全局种子）更灵活。
"""

import torch
g = torch.Generator()        # 创建一个随机数生成器
out = g.manual_seed(42)            # 设置随机种子

print(out)

In [ ]:
"""
random_split 随机数据集划分
"""

from torch.utils.data import Dataset, random_split
import torch

# 一个简单的数据集
class MyDataset(Dataset):
    def __init__(self):
        self.data = list(range(100))  # 假设有 100 条数据

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = MyDataset()

# 设置随机种子（保证每次划分结果相同）
generator = torch.Generator().manual_seed(42)

# 按比例划分 80% 训练集, 20% 验证集
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

print(len(train_dataset), len(val_dataset))  # 输出 80 20
print(train_dataset[0], val_dataset[0])  # 随机划分的样本

80 20
42 2


In [ ]:
"""
分词器的基本使用
"""

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. 加载 tokenizer 和模型
model_checkpoint = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# 2. 输入中文句子
text = "我喜欢机器学习。"

# 3. 编码成模型输入
inputs = tokenizer(
    text,
    return_tensors="pt",  # 返回 PyTorch 张量
    padding=True,
    truncation=True
)

print("编码结果：")
print(inputs)

# 4. 用模型生成翻译
outputs = model.generate(**inputs, max_length=40)

# 5. 解码回人类可读文本
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("翻译结果：", translated_text)

# 6. 批量翻译示例
sentences = ["我喜欢机器学习。", "深度学习很有趣。"]
batch_inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

print("batch 输入token: ", batch_inputs)

batch_outputs = model.generate(**batch_inputs, max_length=40)
batch_translations = [tokenizer.decode(t, skip_special_tokens=True) for t in batch_outputs]

print("批量翻译结果：", batch_translations)

/Users/lisicong/miniforge3/envs/transformer/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


编码结果：
{'input_ids': tensor([[13575, 16183,  2886,     9,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
翻译结果： I like machine learning.
batch 输入token:  {'input_ids': tensor([[13575, 16183,  2886,     9,     0, 65000],
        [    7, 25765,  2886, 27398,     9,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]])}
批量翻译结果： ['I like machine learning.', 'Deep learning is interesting.']


In [9]:
"""
工程上对于truncation的用法是，切分为chunk
长文本 → 切成 chunks → 模型处理 → 结果合并
"""
max_len = 512
stride = 256  # 滑动窗口重叠部分，保证上下文连续


long_text = "我太爱机器学习了！" * 1000
tokens = tokenizer(long_text, return_tensors="pt")["input_ids"][0]
chunks = []
for i in range(0, len(tokens), max_len - stride):
    chunk = tokens[i:i + max_len]
    chunks.append(chunk)

print(len(tokens))
print(len(chunks[-2]))

7001
345


In [ ]:
"""
torch.where
"""

